hyperparameter_ranges = {'lr': ContinuousParameter(0.001, 0.01),
                         'n-epochs': IntegerParameter(100, 200)}<h1> Method 1: Train with custom XGB Container </h1>

This notebook demonstrates how to build and use a custom Docker container for training with Amazon SageMaker that leverages on the <strong>Script Mode</strong> execution that is implemented by the sagemaker-containers library. Reference documentation is available at https://github.com/aws/sagemaker-containers

We start by defining some variables like the current execution role, the ECR repository that we are going to use for pushing the custom Docker container and a default Amazon S3 bucket to be used by Amazon SageMaker.

In [2]:
import boto3
import sagemaker
from sagemaker import get_execution_role

ecr_namespace = 'sagemaker-training-containers/'
prefix = 'script-mode-container-xgb'

ecr_repository_name = ecr_namespace + prefix
role = "arn:aws:iam::342474125894:role/service-role/AmazonSageMaker-ExecutionRole-20190405T234154"
account_id = role.split(':')[4]
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
bucket = sagemaker_session.default_bucket()

print(account_id)
print(region)
print(role)
print(bucket)
print(ecr_repository_name)

342474125894
ap-southeast-1
arn:aws:iam::342474125894:role/service-role/AmazonSageMaker-ExecutionRole-20190405T234154
sagemaker-ap-southeast-1-342474125894
sagemaker-training-containers/script-mode-container-xgb


<h3>Build and push the container</h3>
We are now ready to build this container and push it to Amazon ECR. This task is executed using a shell script stored in the ../script/ folder. Let's take a look at this script and then execute it.

In [3]:
! ../scripts/build_and_push.sh $account_id $region $ecr_repository_name

container  147.3s
 => [8/9] COPY code/* /opt/ml/code/                                        0.1s
 => exporting to image                                                     2.6s
 => => exporting layers                                                    2.6s
[+] Building 219.8s (13/14)                                                     
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 2.29kB                                     0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/ubuntu:16.04            3.3s
 => [auth] library/ubuntu:pull token for registry-1.docker.io              0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 78.18kB                                       0.0s
 => [1/9] FROM docker

<h3>Training with Amazon SageMaker</h3>

Once we have correctly pushed our container to Amazon ECR, we are ready to start training with Amazon SageMaker, which requires the ECR path to the Docker container used for training as parameter for starting a training job.

In [4]:
container_image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest'.format(account_id, region, ecr_repository_name)
print(container_image_uri)

342474125894.dkr.ecr.ap-southeast-1.amazonaws.com/sagemaker-training-containers/script-mode-container-xgb:latest


<h3>Prepare Data</h3>

Now, we upload some dummy data to Amazon S3, in order to define our S3-based training channels.

In [5]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd

X, y = make_classification(
    n_samples=100, n_features=5, n_redundant=0, n_informative=2, n_clusters_per_class=1, n_classes=3
)
print(f"X: {X.shape}, y:{y.shape}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)
train_df = pd.concat([y_train, X_train], axis=1)
train_df.columns = range(train_df.shape[1])
test_df = pd.concat([y_test, X_test], axis=1)
test_df.columns = range(test_df.shape[1])

print(f"train_df: {train_df.shape}, test_df:{test_df.shape}")

X: (100, 5), y:(100,)
train_df: (70, 6), test_df:(30, 6)


In [6]:
train_filename = "../test_data/train/train.csv"
test_filename = "../test_data/val/test.csv"
train_df.to_csv(train_filename, header=True, index=False)
test_df.to_csv(test_filename, header=True, index=False)

train_uri = sagemaker_session.upload_data(train_filename, bucket, prefix + '/train')
test_uri = sagemaker_session.upload_data(test_filename, bucket, prefix + '/val')
print(train_uri)
print(test_uri)
#! rm $train_filename $test_filename

s3://sagemaker-ap-southeast-1-342474125894/script-mode-container-xgb/train/train.csv
s3://sagemaker-ap-southeast-1-342474125894/script-mode-container-xgb/val/test.csv


<h3>Training

Finally, we can execute the training job by calling the fit() method of the generic Estimator object defined in the Amazon SageMaker Python SDK (https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/estimator.py). This corresponds to calling the CreateTrainingJob() API (https://docs.aws.amazon.com/sagemaker/latest/dg/API_CreateTrainingJob.html).

In [7]:
import sagemaker
import json

# Must align with training script argparser
hyperparameters = {
    "hp1": "value1",
    "hp2": 300,
    "hp3": 0.001}

# Custom container can support metric definition
est = sagemaker.estimator.Estimator(container_image_uri,
                                    role,
                                    instance_count=1, 
                                    instance_type='local', # we use local mode
                                    #instance_type='ml.m5.xlarge',
                                    base_job_name=prefix,
                                    hyperparameters=hyperparameters)

train_config = sagemaker.inputs.TrainingInput('s3://{0}/{1}/train/'.format(bucket, prefix), content_type='text/csv')
val_config = sagemaker.inputs.TrainingInput('s3://{0}/{1}/val/'.format(bucket, prefix), content_type='text/csv')
est.fit({'train': train_config, 'validation': val_config })

Creating tmp7egnrajv_algo-1-hf9jk_1 ... 
Attaching to tmp7egnrajv_algo-1-hf9jk_1
algo-1-hf9jk_1  | 2021-01-04 07:46:37,965 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-hf9jk_1  | 2021-01-04 07:46:37,973 sagemaker-containers INFO     Failed to parse hyperparameter hp1 value value1 to Json.
algo-1-hf9jk_1  | Returning the value itself
algo-1-hf9jk_1  | 2021-01-04 07:46:38,004 sagemaker-containers INFO     Failed to parse hyperparameter hp1 value value1 to Json.
algo-1-hf9jk_1  | Returning the value itself
algo-1-hf9jk_1  | 2021-01-04 07:46:38,010 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-hf9jk_1  | 2021-01-04 07:46:38,031 sagemaker-containers INFO     Failed to parse hyperparameter hp1 value value1 to Json.
algo-1-hf9jk_1  | Returning the value itself
algo-1-hf9jk_1  | 2021-01-04 07:46:38,036 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-hf9jk_1  | 2021-01-04 07:46:38,05

## HPO


In [32]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.xgboost.estimator import XGBoost

# Must align with training script argparser
hyperparameters = {
    "hp1": "value1",
    "hp2": 300,
    "hp3": 0.001}

xgb_estimator = sagemaker.estimator.Estimator(container_image_uri,
                                    role,
                                    instance_count=1, 
                                    #instance_type='local', # we use local mode
                                    instance_type='ml.m5.xlarge',
                                    base_job_name=prefix, # Set based job name to easily identify job under HPO
                                    hyperparameters=hyperparameters)

# Must align with training script argparser
hyperparameter_ranges = {'epochs': IntegerParameter(10, 20),
                         #'lr': ContinuousParameter(0.001, 0.01),
                        }

# Must exist in one of the metric definition
objective_metric_name = 'validation-mlogloss'

metric_definitions = [{'Name': 'train-mlogloss',
                       'Regex': 'validation_0-mlogloss:([0-9\\.]+)'},
                     {'Name': 'validation-mlogloss',
                       'Regex': 'validation_1-mlogloss:([0-9\\.]+)'}]

task_tags = [{'Key':'ML Task', 'Value':'XGBoost'}]
tuner = HyperparameterTuner(xgb_estimator,
                            objective_metric_name=objective_metric_name,
                            hyperparameter_ranges=hyperparameter_ranges,
                            metric_definitions=metric_definitions,
                            tags=task_tags,
                            objective_type='Maximize',
                            max_jobs=2,
                            max_parallel_jobs=2)

train_config = sagemaker.inputs.TrainingInput('s3://{0}/{1}/train/'.format(bucket, prefix), content_type='text/csv')
val_config = sagemaker.inputs.TrainingInput('s3://{0}/{1}/val/'.format(bucket, prefix), content_type='text/csv')
tuner.fit({'train': train_config, 'validation': val_config })

............................................!


### Retrieve metrics for best tuning job

In [39]:
from sagemaker.analytics import TrainingJobAnalytics

metric_name = 'validation-mlogloss'
job_name = tuner.best_training_job()
metrics_dataframe = TrainingJobAnalytics(training_job_name=job_name, metric_names=[metric_name]).dataframe()
metrics_dataframe

,timestamp,metric_name,value
0,0.0,validation-mlogloss,1.09765


<h3>check model artifact</h3>
    
make sure it is Booster type,

In [45]:
artifact_path = est.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts']
#artifact_path = "s3://sagemaker-ap-southeast-1-342474125894/script-mode-containe-200820-1621-002-fa80cf0e/output/model.tar.gz"
print(artifact_path)


s3://sagemaker-ap-southeast-1-342474125894/script-mode-containe-200820-1621-002-fa80cf0e/output/model.tar.gz


In [46]:
! aws s3 cp $artifact_path .
! tar -xvf model.tar.gz
! rm model.tar.gz

download: s3://sagemaker-ap-southeast-1-342474125894/script-mode-containe-200820-1621-002-fa80cf0e/output/model.tar.gz to ./model.tar.gz
x model.pth


In [47]:
import pickle
with open('model.pth', 'rb') as f:
    model = pickle.load(f)
    
model

<h1> Method 2: Train with Prebuilt container </h1>

Prebuilt container has both sagemaker-container/sagemaker-inference install in a single image.


In [17]:
from sagemaker.xgboost.estimator import XGBoost

# log group prefix /aws/sagemaker/TrainingJobs

hyperparameters = {
    "hp1": "value1",
    "hp2": 300,
    "hp3": 0.001}

metric_definitions=[
   {'Name': 'custom_train:loss', 'Regex': 'validation_0-mlogloss:=(.*?) '},
   {'Name': 'custom_validation:loss', 'Regex': 'validation_1-mlogloss=(.*?) '}
]
    
xgb_estimator = XGBoost(
    entry_point="train.py",
    source_dir="../docker/code",
    hyperparameters=hyperparameters,
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    #instance_type="local",
    framework_version="1.0-1",
    #metric_definitions=metric_definitions # built in algo does not support metric definition
)

train_config = sagemaker.inputs.TrainingInput('s3://{0}/{1}/train/'.format(bucket, prefix), content_type='text/csv')
val_config = sagemaker.inputs.TrainingInput('s3://{0}/{1}/val/'.format(bucket, prefix), content_type='text/csv')
xgb_estimator.fit({'train': train_config, 'validation': val_config })

2020-08-20 07:45:31 Starting - Starting the training job...
2020-08-20 07:45:33 Starting - Launching requested ML instances......
2020-08-20 07:46:40 Starting - Preparing the instances for training......
2020-08-20 07:48:02 Downloading - Downloading input data
2020-08-20 07:48:02 Training - Downloading the training image...
2020-08-20 07:48:30 Uploading - Uploading generated training model
2020-08-20 07:48:30 Completed - Training job completed
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Invoking user training script.
INFO:sagemaker-containers:Module train does not provide a setup.py. 
Generating setup.py
INFO:sagemaker-containers:Generating setup.cfg
INFO:sagemaker-containers:Generating MANIFEST.in
INFO:sagemaker-containers:Installing module with the following command:
/miniconda3/bin/python -m pip install . 
Processing /opt/ml/code
  

## Deploy from estimator

In [25]:
from sagemaker.serializers import CSVSerializer

serializer = CSVSerializer()
serializer.CONTENT_TYPE = "text/csv"

predictor = xgb_estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.medium",
    serializer=serializer
)


-------------------!

In [ ]:
with open("file.csv") as f:
    payload = f.read()

predictor.predict(payload)

<h3> Get Predictor </h3>

In [33]:
from sagemaker.xgboost.model import XGBoostPredictor
import numpy as np

endpoint_name = "sagemaker-xgboost-2020-08-11-06-52-30-545"
payload = "1,2,3,4,5\n2,3,4,5,6"

xgb_predictor = XGBoostPredictor(endpoint_name)

xgb_predictor.predict(payload)

[['0.335604', '0.32953852', '0.3348575'],
 ['0.3353785', '0.329989', '0.3346325']]

<h3> Low level API - inference </h3>

In [27]:
import boto3

runtime_client = boto3.client('runtime.sagemaker')


payload = "1,2,3,4,5"
endpoint_name = 'sagemaker-xgboost-2020-08-11-06-52-30-545'

response = runtime_client.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType='text/csv',
                                   Body=payload)

result = response['Body'].read().decode('ascii')
print(result)

[[0.3306555449962616, 0.33722642064094543, 0.33211803436279297]]
